<a href="https://colab.research.google.com/github/09Vaibhav/CS_6910/blob/main/solution_03_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl --header "Host: storage.googleapis.com" --header "User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36 Edg/89.0.774.77" --header "Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header "Accept-Language: en-US,en;q=0.9" --header "Referer: https://github.com/google-research-datasets/dakshina" "https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar" -L -o "dakshina_dataset_v1.0.tar"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1915M  100 1915M    0     0   203M      0  0:00:09  0:00:09 --:--:--  190M


In [ ]:
import shutil
shutil.unpack_archive("/content/dakshina_dataset_v1.0.tar",'/content/')

In [ ]:
import re
from tqdm import tqdm
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential,Model,load_model
from keras.layers import Dense,LSTM,GRU,SimpleRNN,input,Dropout,TimeDistributed,RepeatVector,dot,BatchNormalization,concatenate,multiply,Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [ ]:
!pip install wandb

In [ ]:
import wandb
from wandb.keras import WandbCallback

In [ ]:
class seq2seq:
    def __init__(self,cell,emd_size,latent_dmsn,encdr_lyr_size,dcdr_lyr_size,dropouts,epochs,size_of_batch):
        self.cell = cell
        self.emd_size = emd_size
        self.latent_dmsn = latent_dmsn
        self.encdr_lyr_size = encdr_lyr_size
        self.dcdr_lyr_size = dcdr_lyr_size
        self.dropouts=dropouts
        self.epochs=epochs
        self.size_of_batch=size_of_batch

    def get_data(self,path):
        d = pd.read_csv(path,sep="\t",header=None,error_bad_lines=False)
        d = d.dropna()

        dcdr_target_data = np.zeros((d.shape[0],self.max_length_y,self.dcdr_tokens), dtype="float32")

        for i,target_text in enumerate(d[0]):
            target_text = '\t'+target_text+'\n'
            for t, char in enumerate(target_text):
                if t > 0:
                    dcdr_target_data[i, t - 1, self.target_token_index[char]] = 1.0
            dcdr_target_data[i, t:, self.target_token_index["\n"]] = 1.0

        return ([[self.input_token_index[letter] for letter in list('\t'+word+'\n')] for word in d[1]]),\
                ([[self.target_token_index[letter] for letter in list('\t'+word+'\n')] for word in d[0]]),dcdr_target_data

    def create_vocab(self,path):
        d = pd.read_csv(path,sep="\t",header=None,error_bad_lines=False)
        d = d.dropna()

        x = [list('\t'+word+'\n') for word in np.array(d[1])]
        y = [list('\t'+word+'\n') for word in np.array(d[0])]

        telugu_vocab = set()
        english_vocab = set()

        for word in x:
            for char in word:
                english_vocab.add(char)

        for word in y:
            for char in word:
                telugu_vocab.add(char)

        telugu_list = sorted(list(telugu_vocab))
        english_list = sorted(list(english_vocab))

        max_length_x = (np.max([len(i) for i in x]))
        max_length_y = (np.max([len(i) for i in y]))

        return telugu_list,english_list,max_length_x,max_length_y    

    def create_data(self):
        train_path = "/content/dakshina_dataset_v1.0/te/lexicons/te.translit.sampled.train.tsv"
        cv_path = "/content/dakshina_dataset_v1.0/te/lexicons/te.translit.sampled.dev.tsv"
        test_path = "/content/dakshina_dataset_v1.0/te/lexicons/te.translit.sampled.test.tsv"

        telugu_list,english_list,self.max_length_x,self.max_length_y = self.create_vocab(train_path)
        self.encdr_tokens = len(english_list)
        self.dcdr_tokens = len(telugu_list)

        # Dict for char to index
        self.input_token_index = dict([(char, i) for i, char in enumerate(english_list)])
        self.target_token_index = dict([(char, i) for i, char in enumerate(telugu_list)])

        # Dict for index to char
        self.inv_input_token_index = dict({(value,key) for key,value in self.input_token_index.items()})
        self.inv_target_token_index = dict({(value,key) for key,value in self.target_token_index.items()})

        encdr_train,dcdr_train,self.dcdr_target_train = self.get_data(train_path)
        encdr_cv,dcdr_cv,self.dcdr_target_cv = self.get_data(cv_path)
        encdr_test,dcdr_test,self.dcdr_target_test = self.get_data(test_path)


        self.encdr_train = sequence.pad_sequences(encdr_train,maxlen=self.max_length_x,padding="post")
        self.dcdr_train = sequence.pad_sequences(dcdr_train,maxlen=self.max_length_y,padding="post")
        self.encdr_cv = sequence.pad_sequences(encdr_cv,maxlen=self.max_length_x,padding="post")
        self.dcdr_cv = sequence.pad_sequences(dcdr_cv,maxlen=self.max_length_y,padding="post")
        self.encdr_test = sequence.pad_sequences(encdr_test,maxlen=self.max_length_x,padding="post")
        self.dcdr_test = sequence.pad_sequences(dcdr_test,maxlen=self.max_length_y,padding="post")

    def create_model(self):
        encdr_inputs = input(shape=(None,))
        x = Embedding(self.encdr_tokens,self.emd_size,input_length=self.max_length_x)(encdr_inputs)
        for _ in range(self.encdr_lyr_size):
            if self.cell == "lstm":
                x,state_h,state_c = LSTM(self.latent_dmsn,return_state=True,return_sequences=True,dropout=self.dropouts)(x)
                encdr_states = [state_h,state_c]
            elif self.cell == "rnn":
                x,state_c = SimpleRNN(self.latent_dmsn,return_state=True,return_sequences=True,dropout=self.dropouts)(x)
                encdr_states = [state_c]
            elif self.cell == "gru":
                x,state_c = GRU(self.latent_dmsn,return_state=True,return_sequences=True,dropout=self.dropouts)(x)
                encdr_states = [state_c]
                        

        dcdr_inputs = input(shape=(None,))
        dcdr_embb = Embedding(self.dcdr_tokens,self.emd_size,input_length=self.max_length_y)(dcdr_inputs)
        if self.cell == "lstm":
            dcdr_lstm,state_h,state_c = LSTM(self.latent_dmsn, return_sequences=True,return_state=True,dropout=self.dropouts)(dcdr_embb,initial_state=encdr_states)
        elif self.cell == "rnn":
            dcdr_lstm,state_h = SimpleRNN(self.latent_dmsn, return_sequences=True,return_state=True,dropout=self.dropouts)(dcdr_embb,initial_state=encdr_states)
        elif self.cell == "gru":
            dcdr_lstm,state_h = GRU(self.latent_dmsn, return_sequences=True,return_state=True,dropout=self.dropouts)(dcdr_embb,initial_state=encdr_states)

        for i in range(self.dcdr_lyr_size-1):
            if self.cell == "lstm":
                dcdr_lstm,state_h,state_c = LSTM(self.latent_dmsn, return_sequences=True,return_state=True,dropout=self.dropouts)(dcdr_lstm)
            elif self.cell == "rnn":
                dcdr_lstm,state_h = SimpleRNN(self.latent_dmsn, return_sequences=True,return_state=True,dropout=self.dropouts)(dcdr_lstm)
            elif self.cell == "gru":
                dcdr_lstm,state_h = GRU(self.latent_dmsn, return_sequences=True,return_state=True,dropout=self.dropouts)(dcdr_lstm)
        

        dcdr_outputs = TimeDistributed(Dense(self.dcdr_tokens, activation='softmax'))(dcdr_lstm)

        model = Model([encdr_inputs, dcdr_inputs], dcdr_outputs)

        return model

    def percentage_of_correct_test_predictions(self,model):
        count = 0
        for i in tqdm(range(self.encdr_test.shape[0])):
            pred = model([np.expand_dims(self.encdr_test[i],0),np.expand_dims(self.dcdr_test[i],0)])
            actual_out = re.sub('\t|\n','',''.join([self.inv_target_token_index[self.dcdr_test[i][j]] for j in range(self.max_length_y)]))
            predicted_out = re.sub('\t|\n','',''.join(self.inv_target_token_index[np.argmax(pred[0][j])] for j in range(self.max_length_y)))
            if (actual_out==predicted_out):
                count+=1
        return count/self.encdr_test.shape[0]

    def run(self,model):
        # Compile & run training
        model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=["accuracy"])

        model.fit([self.encdr_train,self.dcdr_train], self.dcdr_target_train,
                self.size_of_batch,
                self.epochs,validation_data=([self.encdr_cv,self.dcdr_cv], self.dcdr_target_cv),
                callbacks = [Word_Accuracy_Callback(self),WandbCallback(monitor='WordAccuracy',save_model = True)],verbose=1) 
        


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.7/dist-packages/keras/backend.py", line 4739, in <genexpr>
    ta.write(time, out) for ta, out in zip(output_ta_t, flat_output))  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/tf_should_use.py", line 245, in wrapped
    error_in_function=error_in_function)


In [ ]:
class Word_Accuracy_Callback(tf.keras.callbacks.Callback):
  def __init__(self,obj):
    self.obj = obj

  def on_epoch_end(self,epoch,logs):
    count = 0
    for i in range(self.obj.encdr_cv.shape[0]):
      pred = self.model([np.expand_dims(self.obj.encdr_cv[i],0),np.expand_dims(self.obj.dcdr_cv[i],0)])
      actual_out = re.sub('\t|\n','',''.join([self.obj.inv_target_token_index[self.obj.dcdr_cv[i][j]] for j in range(self.obj.max_length_y)]))
      predicted_out = re.sub('\t|\n','',''.join(self.obj.inv_target_token_index[np.argmax(pred[0][j])] for j in range(self.obj.max_length_y)))
      if (actual_out==predicted_out):
        count+=1

    logs["validation accuracy"] = count/self.obj.encdr_cv.shape[0]
    print("vallidation accuracy =", count/self.obj.encdr_cv.shape[0])

In [ ]:
wandb.init(project='assignment 3 question 2_', entity='vaibhav-kuldeep')

wandb: Currently logged in as: kumar-vaibhav (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


In [ ]:
sweep_config={
    'method': 'random',
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    'parameters':{
        'epochs':{
            'values':[3,5,6]
        },
        'emd_size':{
            'values':[8,12,16,20]
        },
        'encdr_lyr_size':{
            'values':[1,2,3]
        },
        'dcdr_lyr_size':{
            'values':[1,2,3]
        },
        'cell':{
            'values':["rnn","gru","lstm"]
        },
        'dropouts':{
            'values':[0,0.2,0.3,0.4]
        },
        'latent_dmsn':{
            'values':[16,32,64,128,256,512]
        },
        'size_of_batch':{
            'values':[32,64]
        }
    }
}

In [ ]:
!pip install --upgrade wandb
import wandb
!wandb login 7c2b51b201fca423d3798d6c77c45bc79501464a

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
sweep_id = wandb.sweep(sweep_config,project='assignment 3 question 2_', entity="vaibhav-kuldeep")

Create sweep with ID: wbidaosr
Sweep URL: https://wandb.ai/vaibhav-kuldeep/assignment%203%20question%202_/sweeps/wbidaosr


In [ ]:
def train():
    config_defaults={
      'epochs':5,
      'emd_size':16,
      'dropouts':0,
      'encdr_layer_size':2,
      'dcdr_layer_size':2,
      'cell':"lstm",
      'latent_dmsn':512,
      'size_of_batch': 64
       }
    
    wandb.init(config=config_defaults)
    config=wandb.config
    s2s=seq2seq(config.cell,config.emd_size,config.latent_dmsn,config.encdr_lyr_size,config.dcdr_lyr_size,config.dropouts,config.epochs,config.size_of_batch)
    s2s.create_data()
    model=s2s.create_model()
    s2s.run(model)




In [ ]:
wandb.agent(sweep_id,train)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: 20j3h54b with config:
wandb: 	batch_size: 64
wandb: 	cell: rnn
wandb: 	decoder_layers_size: 3
wandb: 	dropouts: 0.2
wandb: 	embedding_size: 20
wandb: 	encoder_layers_size: 3
wandb: 	epochs: 5
wandb: 	latent_dim: 16


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:71: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:72: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.




Epoch 1/5
915/915 [==============================] - 826s 898ms/step - loss: 1.4822 - accuracy: 0.6511 - val_loss: 1.0991 - val_accuracy: 0.7014 - validation accuracy: 0.0000e+00 - _timestamp: 1650544910.0000 - _runtime: 834.0000
Epoch 2/5
915/915 [==============================] - 835s 914ms/step - loss: 1.1237 - accuracy: 0.6863 - val_loss: 0.9991 - val_accuracy: 0.7158 - validation accuracy: 0.0000e+00 - _timestamp: 1650545745.0000 - _runtime: 1669.0000
Epoch 3/5
915/915 [==============================] - 821s 898ms/step - loss: 1.0721 - accuracy: 0.6946 - val_loss: 0.9770 - val_accuracy: 0.7222 - validation accuracy: 0.0000e+00 - _timestamp: 1650546566.0000 - _runtime: 2490.0000
Epoch 4/5
915/915 [==============================] - 822s 900ms/step - loss: 1.0527 - accuracy: 0.7002 - val_loss: 0.9672 - val_accuracy: 0.7241 - validation accuracy: 0.0000e+00 - _timestamp: 1650547389.0000 - _runtime: 3313.0000
Epoch 5/5
915/915 [==============================] - 820s 898ms/step - loss: 

accuracy,▁▆▇▇█
epoch,▁▃▅▆█
loss,█▂▂▁▁
val_accuracy,▁▅▇▇█
val_loss,█▃▂▁▁
validation accuracy,▁▁▁▁▁
accuracy,0.70426
epoch,4
loss,1.0402
val_accuracy,0.72717
val_loss,0.95737


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: ysfmi4tr with config:
wandb: 	batch_size: 64
wandb: 	cell: gru
wandb: 	decoder_layers_size: 1
wandb: 	dropouts: 0.2
wandb: 	embedding_size: 16
wandb: 	encoder_layers_size: 2
wandb: 	epochs: 5
wandb: 	latent_dim: 16


...

wandb: Agent Starting Run: uxd7o5mb with config:
wandb: 	batch_size: 64
wandb: 	cell: gru
wandb: 	decoder_layers_size: 3
wandb: 	dropouts: 0
wandb: 	embedding_size: 8
wandb: 	encoder_layers_size: 1
wandb: 	epochs: 3
wandb: 	latent_dim: 32


Epoch 1/3
915/915 [==============================] - 646s 699ms/step - loss: 1.3931 - accuracy: 0.6592 - val_loss: 1.0907 - val_accuracy: 0.7058 - validation accuracy: 0.0000e+00 - _timestamp: 1650564994.0000 - _runtime: 655.0000
Epoch 2/3
915/915 [==============================] - 636s 695ms/step - loss: 1.0635 - accuracy: 0.6977 - val_loss: 0.9581 - val_accuracy: 0.7246 - validation accuracy: 0.0000e+00 - _timestamp: 1650565630.0000 - _runtime: 1291.0000
Epoch 3/3
915/915 [==============================] - 632s 692ms/step - loss: 0.9993 - accuracy: 0.7121 - val_loss: 0.9189 - val_accuracy: 0.7342 - validation accuracy: 0.0000e+00 - _timestamp: 1650566262.0000 - _runtime: 1923.0000



accuracy,▁▆█
epoch,▁▅█
loss,█▂▁
val_accuracy,▁▆█
val_loss,█▃▁
validation accuracy,▁▁▁
accuracy,0.71212
epoch,2
loss,0.9993
val_accuracy,0.73424
val_loss,0.91889


wandb: Agent Starting Run: staeiegm with config:
wandb: 	batch_size: 32
wandb: 	cell: lstm
wandb: 	decoder_layers_size: 3
wandb: 	dropouts: 0
wandb: 	embedding_size: 20
wandb: 	encoder_layers_size: 2
wandb: 	epochs: 5
wandb: 	latent_dim: 512


Epoch 1/5
1830/1830 [==============================] - 4840s 3s/step - loss: 1.0412 - accuracy: 0.7110 - val_loss: 0.7844 - val_accuracy: 0.7681 - validation accuracy: 0.0000e+00 - _timestamp: 1650571159.0000 - _runtime: 4851.0000
Epoch 2/5
1830/1830 [==============================] - 4966s 3s/step - loss: 0.6442 - accuracy: 0.8047 - val_loss: 0.4943 - val_accuracy: 0.8466 - validation accuracy: 0.0137 - _timestamp: 1650576125.0000 - _runtime: 9817.0000
Epoch 3/5
1830/1830 [==============================] - 4752s 3s/step - loss: 0.3385 - accuracy: 0.8962 - val_loss: 0.3079 - val_accuracy: 0.9059 - validation accuracy: 0.1316 - _timestamp: 1650580876.0000 - _runtime: 14568.0000
Epoch 4/5
 904/1830 [=============>................] - ETA: 31:55 - loss: 0.1933 - accuracy: 0.9412